# Import Google Drive + Darknet Github

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

root_dir = "/content/gdrive/MyDrive/Colab"
base_dir = root_dir + '/Cars'

DN="/content/gdrive/MyDrive/Colab/darknet"
print(DN)

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab/darknet


In [ ]:
!rm $DN -rf

In [ ]:
!git clone https://github.com/AlexeyAB/darknet $DN # Makes a darknet folder

Cloning into '/content/gdrive/MyDrive/Colab/darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.
Updating files: 100% (2056/2056), done.


# Set compiler up (Say if there is a GPU)

In [ ]:
# https://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python

f = open(DN + '/Makefile', 'r')
lines = ''
for line in f:
    if line.find('GPU=0') != -1:
        line = line.replace('GPU=0', 'GPU=1')
    if line.find('CUDNN=0') != -1:
       line = line.replace('CUDNN=0', 'CUDNN=1')
    if line.find('CUDNN_HALF=0') != -1:
       line = line.replace('CUDNN_HALF=0', 'CUDNN_HALF=1')
    if line.find('OPENCV=0') != -1:
        line = line.replace('OPENCV=0', 'OPENCV=1')
#    if line.find('AVX=0') != -1:
#       line = line.replace('AVX=0', 'AVX=1')
#    if line.find('OPENMP=0') != -1:
#        line = line.replace('OPENMP=0', 'OPENMP=1')
    lines += line
f.close()

g = open(DN + '/Makefile', 'w')
g.write(lines)
g.close()

# Compile Darknet

In [ ]:
!echo $DN
!cd $DN; make --silent; clear; echo "Darknet Compiled!"

/content/gdrive/MyDrive/Colab/darknet
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘buff’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
 1443 |         char buff[100];
      |              ^~~~
./src/image_opencv.cpp:1419:9: warning: unused variable ‘it_tb_res’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
 1419 |     int it_tb_res = cv::createTr

# Set Darknet Up


    Batches = number of classes * 2000
    Filters = (number of classes + 5) * 3


In [ ]:
!cp $DN/cfg/yolov3.cfg $DN/cfg/yolov3-train.cfg

f = open(DN + '/cfg/yolov3-train.cfg', 'r')
lines = ''
for line in f:
    if line.find('batch=1') != -1:
        line = line.replace('batch=1', 'batch=32')
    if line.find('subdivisions=1') != -1:
        line = line.replace('subdivisions=1', 'subdivisions=16')
    if line.find('max_batches=500200') != -1:
        line = line.replace('max_batches=500200', 'max_batches=2000')
    if line.find('filters=255') != -1:
        line = line.replace('filters=255', 'filters=18')
    if line.find('classes=80') != -1:
        line = line.replace('classes=80', 'classes=1')
    lines += line
f.close()
with open(DN + '/cfg/yolov3-train.cfg', 'w') as f:
  f.write(lines)

In [ ]:
!cd $DN/data; rm * -fr
# r = recursive, f = force (and shut up)

!unzip -q "$base_dir/cars.zip" -d "$DN/data"
# q = Quiet, d = Target Directory

In [ ]:
OUTPUT=DN + "/backup"
!mkdir "$OUTPUT"
!echo -e "classes = 1\ntrain = $DN/data/train.txt\nvalid = $DN/data/test.txt\nlabels = $DN/data/labels.txt\nnames = $DN/data/cars/classes.txt\nbackup = /content/gdrive/MyDrive/Colab/darknet/backup" > $DN/data/obj.data

mkdir: cannot create directory ‘/content/gdrive/MyDrive/Colab/darknet/backup’: File exists


In [ ]:
!find $DN/data/cars/images/*.jpeg > $DN/data/train.txt
!find $DN/data/cars/images/*.jpeg > $DN/data/test.txt

!mv $DN/data/cars/labels/*.txt $DN/data/cars/images
!find $DN/data/cars/images/*.txt > $DN/data/labels.txt

In [ ]:
!cd $DN; wget https://pjreddie.com/media/files/darknet53.conv.74
#cd $DŃ: wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2024-04-05 11:34:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... ^C


# Run Darknet

In [ ]:
!date
!echo "$OUTPUT/yolov3-train_last.weights"

Mon Apr 22 08:10:25 AM UTC 2024
/content/gdrive/MyDrive/Colab/darknet/backup/yolov3-train_last.weights


In [ ]:
#!cd $DN;./darknet detector  https://stackoverflow.com/questions/63321024/bin-bash-darknet-permission-deniedtrain $DN/data/obj.data $DN/cfg/yolov3-train.cfg /content/darknet/darknet53.conv.74 -dont_show
#!cd $DN; ./darknet detector -dont_show -map train $DN/data/obj.data $DN/cfg/yolov3-train.cfg $DN/darknet53.conv.74
!cd $DN; chmod +x darknet # https://stackoverflow.com/questions/63321024/bin-bash-darknet-permission-denied
!cd $DN; ./darknet detector -i 0 -dont_show -map train data/obj.data cfg/yolov3-train.cfg backup/yolov3-train_last.weights -gpus 0
!date

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.902058), count: 4, class_loss = 0.000000, iou_loss = 0.025742, total_loss = 0.025742 
 total_bbox = 86316, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00

In [ ]:
!date
from IPython.display import display
display('display("Hello World")')
415 workflow runs
Update index.md Deploy Hugo site to Pages #415: Commit efe76ae pushed by hyperagon
main
April 19, 2024 13:04
Queued

print('Not to be confused with print("Hello World") since those don\'t allow copying.')